In [1]:
import numpy as np
import pandas as pd
import cv2
import os

numcam_data = pd.read_csv("D:/TRABAJOS RICARDO/Semestre_13/PFG/Repositorio/manual_annotations/CAMERA.csv", sep= ";")
position_data = pd.read_csv("D:/TRABAJOS RICARDO/Semestre_13/PFG/Repositorio/manual_annotations/DATA.csv", sep= ";")


In [2]:
camara_days = []
position_days = []

# separando el contenido del dataset entre los 3 dias
# primer día: 32 personas, segundo y tercero: 30 personas. Total 92 personas

sub_arrays = np.split(np.transpose(position_data.values), [32*7,62*7])
for i, _ in enumerate(sub_arrays):
    position_days.append(np.transpose(sub_arrays[i]))

sub_arrays = np.split(np.transpose(numcam_data.values), [32,62])
for i, _ in enumerate(sub_arrays):
    camara_days.append(np.transpose(sub_arrays[i]))

#print(camara_days[2].shape)
#print(position_days[2].shape)

In [3]:
# direcciones de entradas y salidas
input_url = "D:/TRABAJOS RICARDO/Semestre_13/PFG/Github/Entrenamiento-Maestro-Estudiante/Acondicionamiento imagenes/Frames/Entradas"
output_url = "D:/TRABAJOS RICARDO/Semestre_13/PFG/Github/Entrenamiento-Maestro-Estudiante/Acondicionamiento imagenes/Frames/Salidas/Day1/Cam1"

# funcion que imprime las etiquetas de los videos
def process_dayx_camx_framex(day2process, cam2process):

    # seleccionando el día (1,2 o 3)
    day_cam_info = pd.DataFrame(camara_days[day2process-1])
    day_position_info = pd.DataFrame(position_days[day2process-1])

    # recorriendo los frames del día
    for frame_num, frame_content in enumerate(day_cam_info.values):
        
        # salto entre frames a analizar
        if frame_num % 20 == 0:
            
            # abriendo imagen del frame
            img = cv2.imread(input_url + "/Day" + str(day2process) + "/Cam" + str(cam2process) + "/frame{:05d}.png".format(frame_num))

            # obteniendo la posicion de las personas durante este frame
            for actual_person, camara_num in enumerate(frame_content):
                
                # comparando si están en la cámara que interesa analizar
                if camara_num == cam2process:
                    
                    # obteniendo la posicion de la persona
                    positions = (day_position_info.values[frame_num,7*actual_person+1:7*actual_person+5])
                    cv2.rectangle(img, (positions[0], positions[1]), (positions[2], positions[3]), (0, 255, 0), 2)
                    # obteniendo la posicion de la persona

            # dibujando la etiqueta
            cv2.imwrite(os.path.join(output_url, 'frame{:05d}.png'.format(frame_num)), img)

process_dayx_camx_framex(day2process=1, cam2process=1)